In [ ]:
using ITensors
using Plots
using Plots.PlotMeasures
using Parameters

### Build Geometry

In [ ]:
# Build 1D chain
# using the Lattice vector is overkill for a chain but makes it easy to
# generalize to other geometries
function chain_nearest_lattices(n; is_periodic = false, neighbor = 1)
  nbonds = n - (is_periodic ? 0 : neighbor)
  chain = Lattice(undef, nbonds)
  for i in 1:nbonds
    chain[i] = LatticeBond(i, ((i + neighbor - 1) % n) + 1)
  end
  chain
end

In [ ]:
# TODO: switch between Heisenberg and Ising Hamiltonians

# struct HamiltonianType{T}
# end

# HamiltonianType(s::AbstractString) = HamiltonianType{Symbol(s)}()

# macro HamiltonianType_str(s)
#   return HamiltonianType{Symbol(s)}
# end

# build_hamiltonian(s::String) = build_hamiltonian(HamiltonianType(s))

### Build the Heisenberg Hamiltonian

$$ H = J_1\sum_i S_i \cdot S_{i + 1} + h\sum_i S_i^z = J_1\bigg(\sum_i S_i^zS_{i+1}^z + \frac{1}{2}S_i^+S_{i+1}^- + \frac{1}{2}S_i^-S_{i+1}^+\bigg) + h\sum_i S_i^z$$

In [ ]:
function build_hamiltonian(sites, nearest::Lattice, J1::Real, next_nearest::Lattice, J2::Real, h::Real)
    ampo = OpSum()

    for b in nearest
        ampo +=       J1, "Sz", b.s1, "Sz", b.s2
        ampo += 0.5 * J1, "S+", b.s1, "S-", b.s2
        ampo += 0.5 * J1, "S-", b.s1, "S+", b.s2
    end

    for b in next_nearest
        ampo +=       J2, "Sz", b.s1, "Sz", b.s2
        ampo += 0.5 * J2, "S+", b.s1, "S-", b.s2
        ampo += 0.5 * J2, "S-", b.s1, "S+", b.s2
    end

    for i in 1:length(sites)
        ampo += h, "Sz", i
    end

    MPO(ampo, sites)
end

### Build the Ising Hamiltonian

$$ H = -J(\sum_i S^z_i \cdot S^z_{i + 1} + g\sum_i S^x_i)$$

In [ ]:
function build_hamiltonian(sites, chain::Lattice, J::Real, g::Real)
  ampo = OpSum()

  for b in chain
      ampo += -J, "Sz", b.s1, "Sz", b.s2
      ampo += -g * J, "Sx", b.s1
  end

  MPO(ampo, sites)
enda

### Run DMRG

In [ ]:
@with_kw struct SweepParams
  num::Int
  dims::Array{Int}
  cutoff::Real
end

@with_kw struct DMRGParams
  initial_bond_dimension::Int
  sweep::SweepParams
end

function run_dmrg(sites, H, ψ₀, params::DMRGParams)
  sweeps = Sweeps(params.sweep.num)
  setmaxdim!(sweeps, params.sweep.dims...)
  setcutoff!(sweeps, params.sweep.cutoff)

  return dmrg(H, ψ₀, sweeps);
end

In [ ]:
"""
Pattern for a parameterized type which allows making literals into types. 
This allows for new types to be defined across the codebase and leveraging 
multiple dispatch.
"""

struct InitialStateType{T}
end

InitialStateType(s::AbstractString) = InitialStateType{Symbol(s)}()

macro InitialStateType_str(s)
  return InitialStateType{Symbol(s)}
end

function build_initial_state(t::InitialStateType"alt", sites)
  state = [isodd(n) ? "Up" : "Dn" for n=1:length(sites)]
  return productMPS(sites, state)
end

function build_initial_state(t::InitialStateType"all up", sites)
  state = fill("Up", length(sites))
  return productMPS(sites, state)
end

function build_initial_state(t::InitialStateType"random", sites)
  return randomMPS(sites, )
end

build_initial_state(s::AbstractString, sites) = build_initial_state(InitialStateType(s), sites)

In [ ]:
@with_kw struct ChainParams
  n::Int
  is_periodic::Bool = false
end

@with_kw struct HeisenbergParams
  chainparams::ChainParams
  dmrgparams::DMRGParams
  sitetype::String
  J1::Real
  J2::Real = 0
  h::Real = 0
  initial_state_type::String = "alt"
end

function make_observations(ψ, params::HeisenbergParams)
  n = params.chainparams.n

  # Sz expectation on each site
  szs = expect(ψ, "Sz")
  szexpplot = plot(szs, title = "S_z expectation")
  ylims!(-0.6, 0.6)
  xlabel!("Site")
  ylabel!("<S_z>")

  # Sz correlation
  cm = correlation_matrix(ψ, "Sz", "Sz")
  qrt = floor(Integer, n/4)
  szcorrplot = plot(abs.(cm[qrt, qrt:(3*qrt)]),  title = "S_z correlation")
  xlabel!("Site")

  println("\nN = $n, J1 = $(params.J1), J2 = $(params.J2)")
  p = plot(szexpplot, szcorrplot, layout = (2, 1), titlefontsize=11, legend=false)
  display(p)
  
  return nothing
end

In [ ]:
function run_heisenberg_model(params:: HeisenbergParams)
  @unpack chainparams, dmrgparams, sitetype, J1, J2, h, initial_state_type = params

  sites = siteinds(sitetype, chainparams.n; conserve_qns = true)

  nearest = chain_nearest_lattices(chainparams.n, is_periodic=chainparams.is_periodic, neighbor=1)
  next_nearest = chain_nearest_lattices(chainparams.n, is_periodic=chainparams.is_periodic, neighbor=2)
  H = build_hamiltonian(sites, nearest, J1, next_nearest, J2, h)

  ψ₀ =  build_initial_state(initial_state_type, sites)
  energy, ψ = run_dmrg(sites, H, ψ₀, dmrgparams)
  
  make_observations(ψ, params)
  
  return ψ
end

# 10 S=1/2 J1

In [ ]:
chainparams = ChainParams(n = 5, is_periodic=false)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 200)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 100)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = -1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, J2 = 0.1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, J2 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1/2", J1 = 1, J2 = 10, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1", J1 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1", J1 = -1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
chainparams = ChainParams(n = 50)
sweepparams = SweepParams(num = 10, dims = [5, 10, 20, 100, 100], cutoff = 1E-8)
dmrgparams = DMRGParams(initial_bond_dimension = 10, sweep=sweepparams)
heisenbergparams = HeisenbergParams(sitetype = "S=1", J1 = 1, J2 = 1, chainparams = chainparams, dmrgparams = dmrgparams)

run_heisenberg_model(heisenbergparams);

In [ ]:
dd